In [1]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import matplotlib.dates as mdates
import os

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the base path to the desired directory on Google Drive
base_path = '/content/drive/MyDrive/Study_1_Data/'

Mounted at /content/drive


In [3]:
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

In [4]:
def process_data(data, columns_to_remove):

    processed_data = data.drop(columns=columns_to_remove).values
    return processed_data

In [5]:
def construct_3d_array(base_dir, participants, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye):
    """
    Construct 3D array from CSV files.
    """
    num_rows = 180  # Define number of rows to keep (last 180 rows)
    arrays_3d = []

    for participant in participants:
        participant_id = f"{int(participant):02d}"  # Format participant number to two digits

        valid_simulations = []

        for simulation in simulations:
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')

            # Check if all files exist
            if all(os.path.exists(file) for file in [hr_file_path, gsr_file_path, head_file_path, eye_file_path]):
                valid_simulations.append(simulation)

        num_valid_simulations = len(valid_simulations)
        if num_valid_simulations == 0:
            continue  # Skip this participant if no valid simulations are found

        array_3d = np.zeros((num_valid_simulations, num_rows, 47)) # hr=1, gsr=1, head=15-3, eye=41-8 total columns after removing columns= 48

        for s_idx, simulation in enumerate(valid_simulations):
            # Process hr data
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            hr_data = read_csv(hr_file_path)
            processed_hr_data = process_data(hr_data, columns_to_remove_hr)
            processed_hr_data = processed_hr_data[-num_rows:]  # Keep only the last 180 rows

            # Process gsr data
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            gsr_data = read_csv(gsr_file_path)
            processed_gsr_data = process_data(gsr_data, columns_to_remove_gsr)
            processed_gsr_data = processed_gsr_data[-num_rows:]  # Keep only the last 180 rows

            # Process head data
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            head_data = read_csv(head_file_path)
            processed_head_data = process_data(head_data, columns_to_remove_head)
            processed_head_data = processed_head_data[-num_rows:]  # Keep only the last 180 rows

            # Process eye data
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')
            eye_data = read_csv(eye_file_path)
            processed_eye_data = process_data(eye_data, columns_to_remove_eye)
            processed_eye_data = processed_eye_data[-num_rows:]  # Keep only the last 180 rows

            # Combine processed data
            combined_data = np.concatenate((processed_hr_data, processed_gsr_data, processed_head_data, processed_eye_data), axis=1)

            array_3d[s_idx, :, :] = combined_data

        arrays_3d.append(array_3d)

    return arrays_3d


In [6]:
sample_size=30
# simulations_train = ['noise','bumps']
# simulations_test=['flat']
# val_indices = [4, 10, 11, 26, 28, 31, 33, 37] # for flat
# train_indices = [0, 1, 2, 3, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 32, 34, 35, 36, 38, 39, 40, 41] # for flat


# simulations_test=['noise']
# simulations_train = ['flat','bumps']
# val_indices = [7, 15, 17, 19, 28, 31, 32, 42, 44, 48] # for noise
# train_indices = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47] # for noise

simulations_test=['bumps']
simulations_train = ['flat','noise']
val_indices = [1, 12, 16, 18, 22, 26, 28, 37, 41] # for speedbumps
train_indices = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 19, 20, 21, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 42, 43, 44] # for speedbumps

In [7]:
participants = [str(i) for i in range(1, 27)]  # Participants 101 to 123
columns_to_remove_hr = []
columns_to_remove_gsr = []
columns_to_remove_eye = ['#Frame','Time', 'Unnamed: 40','ConvergenceValid','Left_Eye_Closed','Right_Eye_Closed','LocalGazeValid','WorldGazeValid']
columns_to_remove_head = ['#Frame','Time', 'Unnamed: 14']

In [8]:
arrays_train = construct_3d_array(base_path, participants, simulations_train, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_test = construct_3d_array(base_path, participants, simulations_test, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)


In [9]:
# Concatenate arrays along the first axis
input_train = np.concatenate(arrays_train, axis=0)
input_test = np.concatenate(arrays_test, axis=0)

# Display the shape of the final concatenated 3D array
print(f"Shape of the final concatenated 3D array: {input_train.shape}")
print(f"Shape of the final concatenated 3D array: {input_test.shape}")

Shape of the final concatenated 3D array: (45, 180, 47)
Shape of the final concatenated 3D array: (23, 180, 47)


In [10]:
def calculate_total_ssq(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    n_columns = [0, 5, 6, 7, 8, 14, 15]
    o_columns = [0, 1, 2, 3, 4, 8, 10]
    d_columns = [4, 7, 9, 10, 11, 12, 13]

    # Calculate sum for each specified set of columns
    n_val = df.iloc[:, n_columns].sum(axis=1)
    o_val = df.iloc[:, o_columns].sum(axis=1)
    d_val = df.iloc[:, d_columns].sum(axis=1)

    return n_val,o_val,d_val

In [11]:
def merge_ssq_column(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          df = pd.read_csv(csv_path)
          # n_val,o_val,d_val = calculate_total_ssq(csv_path)
          # total_ssq_values.append([n_val, o_val, d_val])
          ssq_values_participant = df.iloc[:, 0:17].values.flatten()   # Assuming SSQ values are in columns 1 to 16
          total_ssq_values.append(ssq_values_participant)
  ssq_array = np.array(total_ssq_values)
  return ssq_array

def merge_total_ssq(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq = (n_val+o_val+d_val) * 3.74
          df = pd.read_csv(csv_path)
          df["total-ssq"] = total_ssq
          #print("csv_path: ",csv_path,"   ",total_ssq)
          total_ssq_values.append(total_ssq)
  # Create a DataFrame from the list of total SSQ values
  df_total_ssq = pd.DataFrame(total_ssq_values, columns=["total-ssq"])
  # Convert the list of total SSQ values to a NumPy array
  total_ssq_array = np.array(total_ssq_values)
  return total_ssq_array



In [12]:
output_train=merge_ssq_column(simulations_train,participants)
output_train = np.squeeze(output_train)
output_test=merge_ssq_column(simulations_test,participants)
output_test = np.squeeze(output_test)
output_train_total_ssq=merge_total_ssq(simulations_train,participants)
output_test_total_ssq=merge_total_ssq(simulations_test,participants)
output_train_total_ssq=output_train_total_ssq.reshape(-1, 1)
output_test_total_ssq=output_test_total_ssq.reshape(-1, 1)
print(output_train.shape,output_test.shape,output_train_total_ssq.shape,output_test_total_ssq.shape)
# print(output_train)
# print(output_train_total_ssq)


(45, 16) (23, 16) (45, 1) (23, 1)


In [13]:
import numpy as np
np.set_printoptions(precision=2)
from scipy.stats import spearmanr

stacked_data = np.vstack((output_train, output_test))

# Compute the Spearman correlation
corr, p_value = spearmanr(stacked_data, axis=0)

# Print the correlation matrix
print("Spearman correlation matrix:")
print(corr)
print("P-values matrix:")
print(p_value)
# Compute the Pearson correlation
corr = np.corrcoef(stacked_data, rowvar=False)


# Print the  correlation matrix
print("Pearson correlation matrix:")
print(corr)

Spearman correlation matrix:
[[ 1.00e+00  3.39e-01  3.78e-01  2.85e-01  1.64e-01  4.66e-01  5.03e-01
   7.59e-01  6.59e-02  2.49e-01  2.18e-01  2.17e-01  4.90e-01  3.21e-01
   5.56e-01  1.54e-01]
 [ 3.39e-01  1.00e+00  3.41e-01  3.13e-01  3.92e-01  3.68e-01  1.61e-01
   3.19e-01  3.58e-01  1.90e-01  5.47e-01  1.13e-01  3.02e-01  4.41e-01
   3.40e-01  1.34e-01]
 [ 3.78e-01  3.41e-01  1.00e+00  1.67e-01  8.65e-02  2.66e-01  4.45e-02
   4.11e-01  1.37e-01  6.16e-01  9.35e-02  9.27e-02  2.77e-01  5.85e-02
   3.17e-01  5.39e-02]
 [ 2.85e-01  3.13e-01  1.67e-01  1.00e+00  2.24e-01 -6.38e-04  2.20e-01
   1.88e-01  1.37e-01  3.88e-01  2.63e-01  1.04e-01  1.48e-01  2.49e-01
   1.74e-01  8.65e-02]
 [ 1.64e-01  3.92e-01  8.65e-02  2.24e-01  1.00e+00  2.70e-01  2.90e-01
   3.14e-01  5.85e-01  1.69e-01  4.35e-01  3.05e-01  2.86e-01  4.43e-01
   2.71e-01  2.33e-01]
 [ 4.66e-01  3.68e-01  2.66e-01 -6.38e-04  2.70e-01  1.00e+00  3.66e-01
   5.59e-01  4.57e-01  2.41e-01  5.33e-01  2.10e-01  4.91e-01  4

In [14]:
# import numpy as np


# new_data = np.zeros((38, 3))

# # Assigning original serial numbers to the first column
# new_data[:, 0] = np.arange(1, 39)

# # Assigning original values to the second column
# new_data[:, 1] = output_train[:, 0]

# # Sorting the array based on the values (second column)
# sorted_indices = np.argsort(new_data[:, 1])
# sorted_data = new_data[sorted_indices]

# # Assigning new serial numbers to the third column
# current_val = sorted_data[0][1]
# start_idx = 0
# for i, row in enumerate(sorted_data):
#     if row[1] != current_val:
#         sorted_data[start_idx:i, 2] = np.arange(start_idx + 1, i + 1)
#         start_idx = i
#         current_val = row[1]

# # Assigning new serial numbers for the last group
# sorted_data[start_idx:, 2] = np.arange(start_idx + 1, len(sorted_data) + 1)

# #print(sorted_data)


In [15]:
def scale_input_data(input_train, input_test):
    # Get the shape of the input data
    num_samples_train, time_steps_train, num_features = input_train.shape
    num_samples_test, time_steps_test, _ = input_test.shape

    # Reshape the input data into 2D arrays
    flattened_train_data = input_train.reshape(-1, num_features)
    flattened_test_data = input_test.reshape(-1, num_features)

    # Initialize a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the scaler on the training data and transform both train and test data
    scaled_train_data = scaler.fit_transform(flattened_train_data)
    scaled_test_data = scaler.transform(flattened_test_data)

    # Reshape the scaled data back to its original shape
    scaled_train_data = scaled_train_data.reshape(num_samples_train, time_steps_train, num_features)
    scaled_test_data = scaled_test_data.reshape(num_samples_test, time_steps_test, num_features)

    return scaled_train_data, scaled_test_data

def scale_target_var(target_data):
    min_val, max_val = np.min(target_data, axis=0), np.max(target_data, axis=0)
    target_data = (target_data-min_val)/(max_val-min_val)

    return target_data, min_val, max_val

In [16]:
input_train, input_test= scale_input_data(input_train[:, (60-sample_size):(180-sample_size), :], input_test[:, (60-sample_size):(180-sample_size), :])
# output_train, min_val, max_val = scale_target_var(output_train)

In [17]:
from keras.utils import to_categorical

# Convert the original labels to one-hot encoded format
output_train_encoded = to_categorical(output_train, num_classes=4)

In [18]:

input_val = input_train[val_indices]
input_train = input_train[train_indices]
output_val = output_train_encoded[val_indices]
output_train = output_train_encoded[train_indices]

In [19]:
print("input_train :", input_train.shape)
print("output_train :", output_train.shape)
print("input_val :", input_val.shape)
print("output_val :", output_val.shape)
print("input_test :", input_test.shape)
print("output_test :", output_test.shape)

input_train : (36, 120, 47)
output_train : (36, 16, 4)
input_val : (9, 120, 47)
output_val : (9, 16, 4)
input_test : (23, 120, 47)
output_test : (23, 16)


In [20]:
from keras.layers import Input, Dense, Dropout, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import sklearn
import numpy as np
import tensorflow as tf

np.set_printoptions(precision=2)
total_losses = []

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

def transformer_decoder(inputs, enc_outputs, head_size, num_heads, ff_dim, dropout=0):
    # Self attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    # Cross attention
    x = LayerNormalization(epsilon=1e-6)(res)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, enc_outputs)
    x = Dropout(dropout)(x)
    res = x + res

    # Feed forward
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

def get_transformer(input_shape1, input_shape2):
    enc_inputs = Input(shape=(input_shape1, input_shape2))
    enc_outputs = transformer_encoder(enc_inputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)

    dec_inputs = Input(shape=(1, input_shape2))
    dec_outputs = transformer_decoder(dec_inputs, enc_outputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)

    x = GlobalAveragePooling1D()(dec_outputs)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    # Output layer for 4-class classification with softmax activation
    outputs = Dense(4, activation='softmax')(x)
    # Return the model
    return Model([enc_inputs, dec_inputs], outputs)

for iteration in range(3):
    pred_test_final = np.empty((output_test.shape[0], 0))
    for subclass in range(16):
        # Get input and output layers for Transformer model
        model = get_transformer(120, 47)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        # Create decoder inputs (zeros)
        train_dec_input = np.zeros((input_train.shape[0], 1, input_train.shape[2]))
        val_dec_input = np.zeros((input_val.shape[0], 1, input_val.shape[2]))
        test_dec_input = np.zeros((input_test.shape[0], 1, input_test.shape[2]))

        best_val = 1000000
        patience = 0
        best_model = None

        # Assuming `output_train_encoded` contains one-hot encoded labels
        for k in range(50):
            # Fit the model
            model.fit([input_train, train_dec_input], output_train[:, subclass], epochs=1, batch_size=32)

            pred_val = model.predict([input_val, val_dec_input])

            # Calculate validation loss (categorical crossentropy is already handled in training)
            loss = sklearn.metrics.mean_squared_error(np.argmax(pred_val, axis=1), np.argmax(output_val[:, subclass], axis=1), squared=False)

            if loss <= best_val:
                best_val = loss
                patience = 0
                best_model = model
            else:
                patience += 1
                if patience > 10:
                    print("Early stopping at iteration:", k)
                    break

        # Predict on the test set using the best model
        pred_test = best_model.predict([input_test, test_dec_input])
        pred_test_classes = np.argmax(pred_test, axis=1)
        pred_test_final = np.hstack((pred_test_final, pred_test_classes.reshape(-1, 1)))

    # Summing predictions to calculate total SSQ for each test sample
    pred_total_ssq = []
    for i in range(pred_test.shape[0]):
        total_ssq = 0
        for j in [0, 5, 6, 7, 8, 14, 15]:
            total_ssq += np.sum(pred_test_final[i, j])

        for j in [0, 1, 2, 3, 4, 8, 10]:
            total_ssq += np.sum(pred_test_final[i, j])

        for j in [4, 7, 9, 10, 11, 12, 13]:
            total_ssq += np.sum(pred_test_final[i, j])

        total_ssq = total_ssq * 3.74
        pred_total_ssq.append(total_ssq)

    # Calculate test loss
    loss = sklearn.metrics.mean_squared_error(pred_total_ssq, output_test_total_ssq, squared=False)
    print("Test Loss:", loss)
    total_losses.append(loss)

# Calculate and print average loss over iterations
average_loss = sum(total_losses) / len(total_losses)
print("Average Test Loss:", average_loss)


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.4815 - loss: 1.1773


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (9, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4815 - loss: 1.0675 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5104 - loss: 1.0188 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 1.0537 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5185 - loss: 1.0475 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6053 - loss: 0.9923 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4340 - loss: 0.9208 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5185 - loss: 0.9293
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5579 - loss: 0.9083
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5764 - loss: 0.8549
1/1

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 746ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.4792 - loss: 1.2325


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (9, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 438ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8553 - loss: 0.6039 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8553 - loss: 0.6292 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8553 - loss: 0.6446 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.8657 - loss: 0.4552
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.8553 - loss: 0.5355
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8553 - loss: 0.5619
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.8553 - loss: 0.5114
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.8553 - loss: 0.3995
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8762 - loss: 0.5152 
1/1 ━

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step
Test Loss: 25.329925590786537
Average Test Loss: 25.846917251159084
